In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from yfinance import EquityQuery

from datetime import datetime
from dateutil.relativedelta import relativedelta


pd.set_option('display.max_rows', 50)

In [ ]:
# dat = yf.Ticker("2YY=F")
# dat.info




#US stocks
# exchanges = ['NMS', 'NYQ', 'NGM']
# countries = ['us']

# selected_stocks_dict, ticker_list = get_tickers_stocks(countries, 5000000, exchanges, 120)

# df_us = get_close_prices(ticker_list, period = 2)
# df_us

In [87]:
#Here we first create a filter with EquityQuery and then use it in the yf.screen() function
def get_tickers_stocks(min_dayvolume, exchanges, n):

    #markets = ['region'] + markets
    exchanges = ['exchange'] + exchanges
    
    q = EquityQuery('and', [            #EquityQuery('is-in', markets), #remove the notion of a region
                  EquityQuery('is-in', exchanges),
                  EquityQuery('gt', ['dayvolume', min_dayvolume])
    ])


    response = yf.screen(q, sortField = 'lastclosemarketcap.lasttwelvemonths', sortAsc = False, size=n) #select top 100 companies by market cap


    selected_stocks = {}
    ticker_list = []
    for stock in response['quotes']:
        ticker = stock['symbol']
        ticker_list.append(stock['symbol'])
        selected_stocks[ticker] = {} #initialize the new sub dictionary
        try:
            selected_stocks[ticker]['name'] = stock['shortName']
            selected_stocks[ticker]['type'] = stock['quoteType']
            selected_stocks[ticker]['exchange'] = stock['fullExchangeName']
        except:
            continue

    return selected_stocks, ticker_list





def get_close_prices(ticker_list, period = 2, start = '2022-01-01'):
    
    date_obj = datetime.strptime(start, '%Y-%m-%d')
    end = date_obj + relativedelta(years=period)
    enddate = end.strftime('%Y-%m-%d')

    df = yf.download(ticker_list, start=start, end=enddate)
    df_close = df['Close']

    df_return = df_close.dropna(how='all').copy()

    #Drop tickers that have more that 10% missing data
    df_return = df_return.drop(columns=df_return.columns[df_return.isna().mean() >= 0.1])
        
    #Impute missing data with the previous price (for eu and asia mostly)
    df_cleaned = df_return.ffill().copy()

    
    return df_cleaned

In [90]:
us_exchanges = ['NMS', 'NYQ', 'NGM']
eu_exchanges = ['PAR', 'FRA', 'LSE', 'AMS']
asia_exchanges = ['SHH', 'JPX', 'HKG']

selected_exchanges = us_exchanges + eu_exchanges + asia_exchanges

df_all = pd.DataFrame()
for exchange in selected_exchanges:
    print(f'Extracting from {exchange}')
    exchanges = [exchange]
    selected_stocks_dict, ticker_list = get_tickers_stocks(50000, exchanges, 50)

    if len(ticker_list) > 0: 
        print('YES')
        df = get_close_prices(ticker_list, period = 2, start = '2022-01-01')
        df_all = pd.concat([df_all, df], axis=1)

df_all = df_all.ffill() #ffill again after concatenating the tickers

Extracting from NMS
YES


[*********************100%***********************]  50 of 50 completed

1 Failed download:
['KSPI']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2024-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1641013200, endDate = 1704085200")')
[                       0%                       ]

Extracting from NYQ
YES


[*********************100%***********************]  50 of 50 completed


Extracting from NGM
YES


...

[*********************100%***********************]  50 of 50 completed
[**                     4%                       ]  2 of 50 completed

Extracting from JPX
YES


[*********************100%***********************]  50 of 50 completed
[                       0%                       ]

Extracting from HKG
YES


[*********************100%***********************]  50 of 50 completed

2 Failed downloads:
['0300.HK', '6936.HK']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2024-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1640966400, endDate = 1704038400")')


In [ ]:
df_all